In [1]:
!pip install pyspark==3.2.0
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 19.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805894 sha256=098fe2375a876996b08382b6a4db66389e44b22166d5c8c019adf7d1b8337120
  Stored in directory: /root/.cache/pip/wheels/32/97/d3/8b6d964c8700e4fbb561c71638a92ec55dac9be51eb5fea86d
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-

In [2]:
!pip install pydeequ

In [3]:
import os
import pyspark
os.environ['SPARK_VERSION'] = str(pyspark.__version__)

!echo $SPARK_VERSION

3.2.0


In [4]:
import pydeequ
from pydeequ.profiles import *
from pydeequ.checks import *
from pydeequ.verification import *

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)\
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)\
    .getOrCreate()

In [6]:
spark

In [7]:
df = spark.read.parquet("shop-clients.parquet")
df.show()
df.printSchema()

+--------------------+------+---------+----------+-------------+------------+---+------------+---------------+
|                  id|   sex|     name|   surname|  second_name|passport_num|age|total_orders|           city|
+--------------------+------+---------+----------+-------------+------------+---+------------+---------------+
|0000f59d342448a6a...|  male|    Артем|    Коваль|     Петрович|  8634393217| 64|          20|Санкт-Петербург|
|000389762aa34cf99...|female|    Ольга|    Попова|    Денисович|  1468464122| 64|          14|         Липецк|
|000494226d7149699...|  male|    Денис|     Тупин|     Олегович|  4111709475| 36|          55|           Омск|
|000b516589bf4024a...|female|    Елена|  Ермолова|     Павлович|  7119323175| 64|          76|       Новгород|
|004bb7d7b03a4715a...|  male|Александр|    Иванов|    Денисович|  5511145370| 74|          36|         Москва|
|004bc5df7dfe47a29...|  male|  Дмитрий|    Коваль|     Иванович|  4296589650| 20|          17|         Москва|
|

1) Analyzer

- Size - размер датафрейма
- Completeness - насколько колонка полна (по сути поиск пробелов и null)
- Compliance - проверяет поле на определенное условие, которое мы задаем

In [8]:
from pydeequ.analyzers import *

analyzer = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("name")) \
                    .addAnalyzer(Completeness("surname")) \
                    .addAnalyzer(Completeness("second_name")) \
                    .addAnalyzer(Completeness("passport_num")) \
                    .addAnalyzer(Completeness("city")) \
                    .addAnalyzer(Compliance("age less than 0", 'age<0')) \
                    .addAnalyzer(Compliance("age great than 100", 'age>100')) \
                    .addAnalyzer(Compliance("orders less than 0", 'total_orders<0')) \
                    .run()

analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analyzer)
analysisResult_df.show()

+-------+------------------+------------+-------+
| entity|          instance|        name|  value|
+-------+------------------+------------+-------+
| Column|              city|Completeness|    1.0|
| Column|      passport_num|Completeness|    1.0|
| Column|orders less than 0|  Compliance| 7.5E-4|
| Column|   age less than 0|  Compliance|    0.0|
| Column|       second_name|Completeness|    1.0|
|Dataset|                 *|        Size|20000.0|
| Column|              name|Completeness|    1.0|
| Column|           surname|Completeness|    1.0|
| Column|age great than 100|  Compliance|  0.001|
+-------+------------------+------------+-------+



2) Constraint Verification

Аргументы объекта Check:

- сессия Spark
- уровень проверки, в нашем случае Warning
- наименование чекера

Далее указываются условия

Первый чекер проверяет возраст на ограничение 100 лет, отрицательных значений и отсутствие пропусков.

Второй чекер проверяет наличие ФИО.

Третий чекер проверяет наличие только male и female в столбце age

Четвертый чекер проверяет, что размер датасета не менее 20к записей и уникальность столбца id.

In [9]:
check_age = Check(spark, CheckLevel.Warning, "Users Age Check")\
        .hasMax("age", lambda x: x <= 100.0)  \
        .isNonNegative("age") \
        .isComplete("age")
check_fio = Check(spark, CheckLevel.Warning, "Users FIO Check")\
        .isComplete("name") \
        .isComplete("surname") \
        .isComplete("second_name")
check_sex = Check(spark, CheckLevel.Warning, "Users Sex Check")\
        .isContainedIn("sex", ["male", "female"])
check_dataset = Check(spark, CheckLevel.Error, "Users Dataset Check")\
        .hasSize(lambda x: x >= 20000)\
        .isUnique("id")

Python Callback server started!


In [10]:
checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(check_age) \
    .addCheck(check_fio) \
    .addCheck(check_sex) \
    .addCheck(check_dataset) \
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.toPandas().head(20)

,check,check_level,check_status,constraint,constraint_status,constraint_message
0,Users Age Check,Warning,Warning,"MaximumConstraint(Maximum(age,None))",Failure,Value: 19399.0 does not meet the constraint re...
1,Users Age Check,Warning,Warning,ComplianceConstraint(Compliance(age is non-neg...,Success,
2,Users Age Check,Warning,Warning,"CompletenessConstraint(Completeness(age,None))",Success,
3,Users FIO Check,Warning,Success,"CompletenessConstraint(Completeness(name,None))",Success,
4,Users FIO Check,Warning,Success,"CompletenessConstraint(Completeness(surname,No...",Success,
5,Users FIO Check,Warning,Success,CompletenessConstraint(Completeness(second_nam...,Success,
6,Users Sex Check,Warning,Success,ComplianceConstraint(Compliance(sex contained ...,Success,
7,Users Dataset Check,Error,Success,SizeConstraint(Size(None)),Success,
8,Users Dataset Check,Error,Success,"UniquenessConstraint(Uniqueness(List(id),None))",Success,


---

In [12]:
!pip install great_expectations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.9 MB/s eta 0:00:00
